# The Data Cleaning Notebook

This notebook documents the cleaning process for the Fifa 2019 Data. It creates a new csv file in ./data/out/clean.csv

## Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from collections import Counter as counter

## Load Data to a data table

In [2]:
df_fifa = pd.read_csv("../data/data.csv")

# Manipulation

## Convert the value and wage into proper currency

In [3]:
def value_to_int(df_value):
    try:
        value = float(df_value[1:-1]) # This return 110.5 from €110.5M
        suffix = df_value[-1:] # This return M or K
        if suffix == 'M':
            value = value * 1000000
        elif suffix == 'K':
            value = value * 1000
    except:
        value = 0
    return value

df_fifa['Value'] = df_fifa['Value'].apply(value_to_int)
df_fifa['Wage'] = df_fifa['Wage'].apply(value_to_int)
df_fifa['Release Clause'] = df_fifa['Release Clause'].apply(value_to_int)

## Convert the height to CM

In [4]:
# Inch = 2.54 CM
# Foot = 2.54*12 = 30.48
def convert_to_cm(df_value):
    height = 0
    try:
        feet,inches = str(df_value).split("'",)
        feet = eval(feet)
        inches = eval(inches)
        height = 30.48*feet + 2.54*inches
    except:
        pass #do nothing
    return int(height)

df_fifa['Height'] = df_fifa['Height'].apply(convert_to_cm)

## Clean weight data

In [5]:
def remove_lbs(df_value):
    try:
        weight = int(df_value[0:-3])
    except:
        weight = 0
    return weight

df_fifa['Weight'] = df_fifa['Weight'].apply(remove_lbs)

## Cycle through skill columns and add them up

In [6]:
def evaluate_the_row(x):
    try:
        return eval(x)
    except:
        return 0

# 26 Positions need addition
for i in range(28,54):
    df_fifa.iloc[:,i] = df_fifa.iloc[:,i].apply(evaluate_the_row)


## Remove Cells where key items are 0

In [7]:
df_fifa = df_fifa[df_fifa.Value != 0]
df_fifa = df_fifa[df_fifa.Overall != 0]
df_fifa = df_fifa[df_fifa.Height != 0]
df_fifa = df_fifa[df_fifa.Weight != 0]

## Add new column: Create a variable with a classified position

In [8]:
def classify_position(df_value):
    if(df_value == 'GK'):
        return 1
    elif(df_value in ['RCB', 'CB', 'LCB', 'LB', 'RB', 'RWB', 'LWB']):
        return 2
    elif(df_value in ['RCM', 'LCM', 'LDM', 'CDM', 'CAM', 'RM', 'LAM', 'LM', 'RDM', 'CM', 'RAM']):
        return 3
    elif(df_value in ['RF', 'LF', 'ST', 'LW', 'RS', 'LS', 'RW', 'CF']):
        return 4
    return 0

df_fifa['PositionCode'] = df_fifa['Position'].apply(classify_position)

## Create Binary Variables for new Position

In [9]:
def goalkeeper_position(df_value):
    if (df_value == 1):
        return 1
    return 0
df_fifa['Is_GK'] = df_fifa['PositionCode'].apply(goalkeeper_position)

def defense_position(df_value):
    if (df_value == 2):
        return 1
    return 0
df_fifa['Is_Defense'] = df_fifa['PositionCode'].apply(defense_position)

def midfield_position(df_value):
    if (df_value == 3):
        return 1
    return 0
df_fifa['Is_Midfield'] = df_fifa['PositionCode'].apply(midfield_position)

def offense_position(df_value):
    if (df_value == 4):
        return 1
    return 0
df_fifa['Is_Offense'] = df_fifa['PositionCode'].apply(offense_position)

## Create Binary Variables for From England

In [10]:
def from_england(df_value):
    if (df_value == "England"):
        return 1
    return 0
df_fifa['From_England'] = df_fifa['Nationality'].apply(from_england)

## Create Binary Variables for from Premier league team

In [16]:
def in_premier(df_value):
    if (df_value in['Manchester United','Manchester City','Chelsea','Tottenham Hotspur','Liverpool','Arsenal','Manchester United','Leicester City','Everton','West Ham United','Wolverhampton Wanderers','Crystal Palace','Fulham','Watford','Burnley','Southampton','Bournemouth','Brighton & Hove Albion','Newcastle United','Huddersfield Town','Cardiff City']):
        return 1
    return 0
df_fifa['In_Premier'] = df_fifa['Club'].apply(in_premier)

# Error Checking

## Reviewing Value

In [11]:
#df_fifa['Value'].describe().apply(lambda x: format(x, 'f'))

## Reviewing Wage

In [12]:
#df_fifa['Wage'].describe().apply(lambda x: format(x, 'f'))

## Check Positions were added correctly

In [13]:
#df_fifa.iloc[:,28:54]

# Visualizations

## Corelation Matrix

In [14]:
#corrmat = df_fifa.corr()
#f, ax = plt.subplots(figsize=(12,9))
#sns.heatmap(corrmat, vmax=.8, square=True)

# Write to CSV

In [30]:
# Clean
df_fifa.to_csv(r'../data/clean.csv', index=None, header=True)
df_fifa.loc[df_fifa['In_Premier'] != 1].to_csv(r"../data/butpremier.csv", index=None, header=True)

# Export the teams
for name in ['Manchester United','Manchester City','Chelsea','Tottenham Hotspur','Liverpool','Arsenal','Manchester United','Leicester City','Everton','West Ham United','Wolverhampton Wanderers','Crystal Palace','Fulham','Watford','Burnley','Southampton','Bournemouth','Brighton & Hove Albion','Newcastle United','Huddersfield Town','Cardiff City']:
    df_fifa.loc[df_fifa['Club'] == name].to_csv(r"../data/Premier League Teams/"+name+".csv", index=None, header=True)
